In [4]:
#XML內容抓取
# 235/1396133
import codecs
import os
import xml.etree.ElementTree as ET
import time
import sys

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = 'C:\\Users\\user\\Desktop\\SNA\\meeting\\source\\SCS_4.0\\'

file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

two_verb = {} 
    
error_num = 0
much = 0
line_index = 0

for file in file_list:
    
    tree = ET.parse(file_path+file)
    root = tree.getroot()

    for sentence in root.iter('sentence'):
        line_index = line_index + 1
        try:
            temp = sentence.text.split()

            verb_list = []
            
            for i in temp:
                word = i.split('(')
                word[1] = '('+word[1].split('[')[0]

                if word[0] in modalverb:
                    verb_list.append(''.join(word))

            if len(verb_list) >= 2:
                if len(verb_list) >= 3:
                    much = much + 1
                '''for l in range(len(verb_list)):
                    for k in range(len(verb_list)-l-1):
                        k = k + l + 1
                        if (verb_list[l],verb_list[k]) in two_verb:
                            two_verb[(verb_list[l],verb_list[k])] = two_verb[(verb_list[l],verb_list[k])] + 1
                        else:
                            two_verb[(verb_list[l],verb_list[k])] = 1'''
                for l in range(len(verb_list)-1):
                    k = l + 1
                    if (verb_list[l],verb_list[k]) in two_verb:
                        two_verb[(verb_list[l],verb_list[k])] = two_verb[(verb_list[l],verb_list[k])] + 1
                    else:
                        two_verb[(verb_list[l],verb_list[k])] = 1
     
        except:
            '''e = sys.exc_info()[0]
            print (e)
            time.sleep(0.3)'''
            error_num = error_num + 1

pop = []

#是否照順序
'''for i in two_verb:
    if (i[1],i[0]) in two_verb and i not in pop and i[0] != i[1]:
        two_verb[i] = two_verb[i] + two_verb[(i[1],i[0])]
        pop.append((i[1],i[0]))

print (len(two_verb)) #230/614
print (len(pop)) #101/188
for j in pop:
    two_verb.pop(j)'''

two_verb = sorted(two_verb.items(), key=lambda d:d[1], reverse = True)

print (error_num) #235
print (much) #465
print (len(two_verb)) #129(2個,無前後)/426(2個,有前後)/641(多個,有前後)
print (line_index)

for i in two_verb:
    print (i[0],i[1])

235
465
629
1396133
('要(D)', '能(D)') 452
('必須(D)', '要(D)') 340
('要(D)', '要(D)') 286
('會(D)', '得(DE)') 282
('應該(D)', '要(D)') 259
('能(D)', '得(DE)') 259
('可以(D)', '得(DE)') 210
('要(D)', '得(DE)') 197
('會(D)', '會(D)') 189
('應(D)', '可(D)') 187
('必須(D)', '能(D)') 166
('能(D)', '能(D)') 164
('可(D)', '可(D)') 163
('應該(D)', '會(D)') 155
('應該(D)', '可以(D)') 148
('要(D)', '會(D)') 147
('要(D)', '當(VG)') 118
('當(P)', '要(D)') 112
('可以(D)', '可以(D)') 109
('會(D)', '要(D)') 99
('得(DE)', '得(DE)') 97
('需要(VK)', '能(D)') 71
('該(Nes)', '會(Nc)') 70
('要(D)', '可以(D)') 69
('該(Nes)', '會(Na)') 69
('應(D)', '能(D)') 66
('要(D)', '能夠(D)') 66
('得(DE)', '要(D)') 64
('應(D)', '會(D)') 62
('可(D)', '得(VJ)') 61
('可(D)', '得(D)') 60
('該(Nes)', '可(D)') 58
('可以(D)', '當(VG)') 53
('可(D)', '得(DE)') 52
('可(D)', '該(Nes)') 51
('該(Nes)', '該(Nes)') 50
('該(Nes)', '會(D)') 48
('會(D)', '能(D)') 46
('應該(D)', '能(D)') 46
('能(D)', '會(D)') 45
('要(D)', '可(D)') 45
('得(D)', '要(D)') 45
('該(Nes)', '應(D)') 44
('應(D)', '該(Nes)') 43
('當(P)', '能(D)') 42
('能(D)', '可(D)'

In [4]:
#情態動詞存入DB內
# 235/1396133
import os
import xml.etree.ElementTree as ET
import sys
import time
import pymysql

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = '../../desktop/SCS_4.0/'
connection = pymysql.connect(host="127.0.0.1", user="redwing", passwd="", db="modalverb",charset='utf8')

file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

two_verb = {} 
    

with connection.cursor() as cursor:
    
    sql = "INSERT into verb_in_sentence(two_verb,two_pos,verb_index,file_sentence) value(\"{}\",\"{}\",\"{}\",\"{}\")"
    
    for file in file_list:

        tree = ET.parse(file_path+file)
        root = tree.getroot()

        slen = 0 #句子數目計算

        for sentence in root.iter('sentence'):

            try:
                temp = sentence.text.split()

                verb_list = []  #情態動詞陣列
                verb_index = []  #情態動詞所在位置陣列
                vi = 0 #index計算

                for i in temp:

                    word = i.split('(')
                    word[1] = '('+word[1].split('[')[0]

                    if word[0] in modalverb:
                        verb_list.append(''.join(word))
                        verb_index.append(vi)

                    vi = vi + 1

                if len(verb_list) >= 2:
                    for l in range(len(verb_list)):
                        for k in range(len(verb_list)-l-1):
                            k = k + l + 1

                            com1 = verb_list[l].split('(')
                            com2 = verb_list[k].split('(')

                            verb1 = com1[0]
                            verb2 = com2[0]
                            index1 = str(verb_index[l])
                            index2 = str(verb_index[k])
                            pos1 = '('+com1[1]
                            pos2 = '('+com2[1]
                            
                            cursor.execute(sql.format(verb1+','+verb2 ,pos1+','+pos2 ,index1+','+index2 ,file+'@'+str(slen)))
                            
                            
                            '''print (verb1,verb2,index1,index2,pos1,pos2,file+'@'+str(slen))
                            time.sleep(0.3)'''
            except:
                '''e = sys.exc_info()[0]
                print (e)
                time.sleep(0.3)'''
                continue
            finally:
                slen = slen + 1

connection.commit()
connection.close()
print ('OK')

OK


In [2]:
#情態動詞存入DB內(修改)
# 235/1396133
# 8744/9285/9939 包含情態動詞的不重複句子數/包含情態動詞的重複句子數(A,B)(B,C)/包含情態動詞的完整重複句子數(A,B)(B,C)(A,C)
# 4928/9285 詞性為(D),(D)/同上
import os
import xml.etree.ElementTree as ET
import sys
import time
import pymysql

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = '../../desktop/SCS_4.0/'
connection = pymysql.connect(host="127.0.0.1", user="redwing", passwd="", db="modalverb",charset='utf8')

file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

two_verb = {} 
match_line = 0  #情緒動詞句子數
match_line2 = 0 #情緒動詞句子數(重複)

with connection.cursor() as cursor:
    
    sql = "INSERT into verb_in_sentence(two_verb,two_pos,verb_index,file_sentence) value(\"{}\",\"{}\",\"{}\",\"{}\")"
    
    for file in file_list:
        
        verbline_in_file = 0 #每個file有情緒動詞句子數

        tree = ET.parse(file_path+file)
        root = tree.getroot()

        slen = 0 #句子數目計算

        for sentence in root.iter('sentence'):

            try:
                temp = sentence.text.split()

                verb_list = []  #情態動詞陣列
                verb_index = []  #情態動詞所在位置陣列
                vi = 0 #index計算

                for i in temp:

                    word = i.split('(')
                    word[1] = '('+word[1].split('[')[0]

                    if word[0] in modalverb:
                        verb_list.append(''.join(word))
                        verb_index.append(vi)

                    vi = vi + 1

                if len(verb_list) >= 2:
                    match_line = match_line + 1
                    verbline_in_file = verbline_in_file + 1
                    for l in range(len(verb_list)-1):
                        k = l + 1
                        match_line2 = match_line2 + 1
                        com1 = verb_list[l].split('(')
                        com2 = verb_list[k].split('(')

                        verb1 = com1[0]
                        verb2 = com2[0]
                        index1 = str(verb_index[l])
                        index2 = str(verb_index[k])
                        pos1 = '('+com1[1]
                        pos2 = '('+com2[1]
                            
                        cursor.execute(sql.format(verb1+','+verb2 ,pos1+','+pos2 ,index1+','+index2 ,file+'@'+str(slen)))
                            
                            
                        #print (verb1,verb2,index1,index2,pos1,pos2,file+'@'+str(slen))
                        #time.sleep(1)
                    #print (temp)
            except:
                '''e = sys.exc_info()[0]
                print (e)
                time.sleep(0.3)'''
                continue
            finally:
                slen = slen + 1
    
        print (file,verbline_in_file)
    
connection.commit()
connection.close()

print (match_line)
print (match_line2)
print ('OK')

asbc_a_001.xml 30
asbc_a_002.xml 35
asbc_a_003.xml 15
asbc_a_004.xml 58
asbc_a_005.xml 135
asbc_a_006.xml 193
asbc_a_007.xml 105
asbc_a_008.xml 50
asbc_a_009.xml 80
asbc_a_010.xml 150
asbc_a_011.xml 50
asbc_a_012.xml 56
asbc_a_013.xml 124
asbc_a_014.xml 19
asbc_a_015.xml 12
asbc_a_016.xml 30
asbc_a_017.xml 19
asbc_a_018.xml 10
asbc_a_019.xml 12
asbc_a_020.xml 7
asbc_a_021.xml 16
asbc_a_022.xml 15
asbc_a_023.xml 5
asbc_a_024.xml 12
asbc_a_025.xml 26
asbc_a_026.xml 16
asbc_a_027.xml 11
asbc_a_028.xml 18
asbc_a_029.xml 27
asbc_a_030.xml 19
asbc_a_031.xml 28
asbc_a_032.xml 25
asbc_a_033.xml 30
asbc_a_034.xml 51
asbc_a_035.xml 16
asbc_a_036.xml 15
asbc_a_037.xml 5
asbc_a_038.xml 10
asbc_a_039.xml 13
asbc_a_040.xml 14
asbc_a_041.xml 14
asbc_a_042.xml 9
asbc_a_043.xml 12
asbc_a_044.xml 62
asbc_a_045.xml 33
asbc_a_046.xml 20
asbc_a_047.xml 34
asbc_a_048.xml 56
asbc_a_049.xml 29
asbc_a_050.xml 51
asbc_a_051.xml 125
asbc_a_052.xml 116
asbc_a_053.xml 108
asbc_a_054.xml 97
asbc_a_055.xml 76
asbc_a

In [2]:
#DB句子查詢
# 235/1396133
import os
import xml.etree.ElementTree as ET
import time
import pymysql
from colorama import init

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = '../../desktop/SCS_4.0/'
connection = pymysql.connect(host="127.0.0.1", user="redwing", passwd="", db="modalverb",charset='utf8')

file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

try:
    with connection.cursor() as cursor:
        
        sql = 'select `file_sentence`,`verb_index`,`two_pos` from `verb_in_sentence` where `two_verb` = "{}"'
        
        ch = '得,得'
        pos = '(D),(D)'
        
        cursor.execute(sql.format(ch))
        
        for i in cursor:
            
            '''#選擇何種詞性
            if i[2] != pos:
                continue'''
            
            index = i[0].split('@')

            tree = ET.parse(file_path+index[0])
            root = tree.getroot()
            
            sentence = root.findall('./article/text/sentence')[int(index[1])].text
            
            temp = ''
            
            count = 0
            for j in sentence.split():
                vi = [int(i[1].split(',')[0]),int(i[1].split(',')[1])] #verb index
                if count in vi:
                    temp = temp + '\033[31;46m' + j + '\033[0m'
                else:
                    temp = temp + j#j.split('(')[0]
                count = count + 1
                    
            sentence = temp
            
            '''#印出前後句
            sentence_pre = 'head'
            sentence_next = 'end'
            
            if int(index[1]) > 0:
                sentence_pre = root.findall('./article/text/sentence')[int(index[1])-1].text
            if int(index[1]) < len(root.findall('./article/text/sentence'))-1:
                sentence_next = root.findall('./article/text/sentence')[int(index[1])+1].text'''
            
            #print (sentence_pre)
            print (sentence)
            #print (sentence_next)
            
            print (i[1])
            print ()
            time.sleep(0.3)
finally:
    connection.close()

因為(Cbb)觀瀾(Nb)這(Nep)名字(Na)取(VC)得(DE)好(VH)所以(Cbb)只要(Cbb)看(VE)[+vrv]得(DE)到(VE)[+vrr]湖(Na)邊(D)愛(VL)談(VE)什麼(Nep)都(D)好像(D)順理成章(VH)；(SEMICOLONCATEGORY)
5,10

只不過(Cbb)有(V_2)人(Na)浮(VA)得(DE)高(VH)、(PAUSECATEGORY)有(V_2)人(Na)沈(VH)得(DE)深(VH)，(COMMACATEGORY)
4,10

修(VC)得(DE)安民(VA)云云(T)說(VE)得(DE)過於(Dfa)堂皇(VH)而(Caa)蹈空(VA)，(COMMACATEGORY)
1,5

來(VA)得(D)快(VH)去(VCL)得(D)也(D)快(VH)。(PERIODCATEGORY)
1,4

得(D)得(D)得(D)，(COMMACATEGORY)
0,1

得(D)得(D)得(D)，(COMMACATEGORY)
1,2

得(D)得(D)得(D)…(ETCCATEGORY)
0,1

得(D)得(D)得(D)…(ETCCATEGORY)
1,2

得(D)得(D)得(D)，(COMMACATEGORY)
0,1

得(D)得(D)得(D)，(COMMACATEGORY)
1,2

得(D)得(D)得(D)…(ETCCATEGORY)
0,1



KeyboardInterrupt: 

In [1]:
#從DB查詢某verb其詞性各別出現頻率
# 235/1396133
import os
import xml.etree.ElementTree as ET
import time
import pymysql

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = '../../desktop/SCS_4.0/'
connection = pymysql.connect(host="127.0.0.1", user="redwing", passwd="", db="modalverb",charset='utf8')

file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

try:
    with connection.cursor() as cursor:
        
        sql = 'select `two_pos` from `verb_in_sentence` where `two_verb` = "{}"'
        
        ch = '要,能'
        
        cursor.execute(sql.format(ch))
        
        pos_num = {}
        
        psum = 0
        
        for i in cursor:
            psum = psum + 1
            if i[0] not in pos_num:
                pos_num[i[0]] = 1
            else:
                pos_num[i[0]] = pos_num[i[0]] + 1
        
        pos_num = sorted(pos_num.items(), key=lambda d:d[1], reverse = True)
        
        print (ch,len(pos_num),psum)
        for i in pos_num:
            print (i[0],i[1])
        
finally:
    connection.close()

要,能 5 501
(D),(D) 469
(VE),(D) 16
(VC),(D) 8
(D),(VL) 7
(Cbb),(VL) 1


In [ ]:
#DB句子查詢(修改)
# 235/1396133
import os
import xml.etree.ElementTree as ET
import time
import pymysql
from colorama import init

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = '../../desktop/SCS_4.0/'
connection = pymysql.connect(host="127.0.0.1", user="redwing", passwd="", db="modalverb",charset='utf8')

file_list = []

for file in os.listdir(file_path):
    file_list.append(file)

try:
    with connection.cursor() as cursor:
        
        sql = 'select `file_sentence`,`verb_index`,`two_pos` from `verb_in_sentence` where `two_verb` = "{}"'
        
        ch = '得,得'
        pos = '(D),(D)'
        
        cursor.execute(sql.format(ch))
        
        for i in cursor:
            
            '''#選擇何種詞性
            if i[2] != pos:
                continue'''
            
            index = i[0].split('@')

            tree = ET.parse(file_path+index[0])
            root = tree.getroot()
            
            sentence = root.findall('./article/text/sentence')[int(index[1])].text
            
            temp = ''
            
            count = 0
            for j in sentence.split():
                vi = [int(i[1].split(',')[0]),int(i[1].split(',')[1])] #verb index
                if count in vi:
                    temp = temp + '\033[31;46m' + j + '\033[0m'
                else:
                    temp = temp + j#j.split('(')[0]
                count = count + 1
                    
            sentence = temp
            
            '''#印出前後句
            sentence_pre = 'head'
            sentence_next = 'end'
            
            if int(index[1]) > 0:
                sentence_pre = root.findall('./article/text/sentence')[int(index[1])-1].text
            if int(index[1]) < len(root.findall('./article/text/sentence'))-1:
                sentence_next = root.findall('./article/text/sentence')[int(index[1])+1].text'''
            
            #print (sentence_pre)
            print (sentence)
            #print (sentence_next)
            
            print (i[1])
            print ()
            time.sleep(0.3)
finally:
    connection.close()

In [1]:
import pymysql
from ipywidgets import IntProgress
from IPython.display import display

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
connection = pymysql.connect(host="140.119.164.170", user="salan40319", passwd="104753018", db="SinicaCorpus",charset='utf8')

word = '會'

p = IntProgress()
p.max = 54580
p.description = 'start'
display(p)

try:
    with connection.cursor() as cursor:
        sql = "select `id` from `corpus_segpos` where `word` = '{}'"
        sql2 = "select * from `corpus_segpos` where `id` = '{}'"

        words = [] #27290
        idd = []
        check = False
        pairs = []
        index = 0

        cursor.execute(sql.format(word))

        for i in cursor:
            words.append(i[0])
            if i[0] != 1:
                idd.append(i[0]-1)

        pair = ''
        for i in sorted(words+idd):
            cursor.execute(sql2.format(i))
            for j in cursor:
                pair = pair + j[1].decode('utf8') + j[2].decode('utf8')
            if check:
                pairs.append(pair)
                check = False
                pair = ''
            else:
                check = True

            index = index + 1
            p.value = index
            p.description = str(index)

        print (pairs[:10])
        
except:
    print ('error')
finally:
    p.description = 'end'
    connection.close()

error


In [2]:
import os
import xml.etree.ElementTree as ET
from ipywidgets import IntProgress
from IPython.display import display
import codecs

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
file_path = "C:\\Users\\user\\Desktop\\SNA\\meeting\\source\\SCS_4.0\\"
out_path = "C:\\Users\\user\\Desktop\\"

file_list = []
count = 0

for file in os.listdir(file_path):
    file_list.append(file)
    
p = IntProgress()
p.max = len(file_list)
p.description = 'start'
display(p)
    
    
dic = {}
sen = {}

for file in file_list:

    tree = ET.parse(file_path+file)
    root = tree.getroot()

    for sentence in root.iter('sentence'):
        try:
            temp = sentence.text.split()

            index = 0
            pairs = []

            for i in temp:
                words = i.split('(')
                words[1] = '(' + words[1]

                if words[0] in modalverb:
                    if index != len(temp)-1:
                        if (words[1] == '(D)' or words[1][1] == 'V') and \
                            (temp[index+1].split('(')[1][0] == 'V' or temp[index+1].split('(')[1][0] == 'D' \
                            or temp[index+1].split('(')[1][0] == 'P' or temp[index+1].split('(')[1][0] == 'C'):
                            pairs.append(temp[index])
                index = index + 1

            for i in pairs:
                for j in range(len(pairs)-1):    
                    term = pairs[j]+'_'+pairs[j+1]
                
                    if term not in dic:
                        dic[term] = 1
                    else:
                        dic[term] += 1

                    if term not in sen:
                        sen[term] = [''.join(temp)]
                    elif len(sen[term]) < 3:
                        if ''.join(temp) not in sen[term]:
                            sen[term].append(''.join(temp))
        except:
            with codecs.open("C:\\Users\\user\\Desktop\\error.txt",'ab','utf8') as f:
                try:
                    f.write(file+'\t'+sentence.text+'\r\n')
                except:
                    f.write(file+'\tNone\r\n')
                    
    count = count + 1
    p.value = count
    p.description = str(count)

dic = sorted(dic.items(), key=lambda d:d[1], reverse = True)
with codecs.open(out_path+'output.txt','wb','utf8') as f:
    for i in dic:
        #print (i[0],i[1])
        f.write(str(i[0])+' '+str(i[1]))
        '''for j in sen[i[0]]:
            f.write(j+'\r\n')'''
        f.write('\r\n')
        
p.description = 'end'